In [2]:
%load_ext autoreload

In [1]:
from datetime import date
from glob import glob

import pandas as pd

In [2]:
%autoreload 2

from src.misc import get_country_alpha2

In [3]:
# Get definitions
%run "00-definitions.ipynb"

## Read installed capacity data for wind power

Read capacity data from externally downloaded files by WindEurope. Each file has data for one year.

In [4]:
frames = list()
for filepath in glob("../data/external/Cumulative_capacity_table_data_*.csv"):
    frames.append(pd.read_csv(filepath))
wind_capacity_data = pd.concat(frames)

Find country codes for the capacity data and set the date of each record to the beginning of the next year.

In [5]:
# Set the country codes
wind_capacity_data["Country code"] = wind_capacity_data["Country"].apply(get_country_alpha2)

# Set dates to beginning of the year
wind_capacity_data["Date"] = wind_capacity_data["Year"].apply(
    lambda y: date(y + 1, 1, 1)
)

# Make a data frame with installed cap. at the beginning of the year
selected_country_codes = list(AREAS.values())
_df = wind_capacity_data.set_index(['Country code', 'Date']).loc[selected_country_codes]
wind_cap = pd.DataFrame({'Wind Onshore': _df['Cumulative onshore capacity'],
                         'Wind Offshore': _df['Cumulative offshore capacity']}).unstack(0)

### Data from other sources

Patching data with data from ENTSO-E for Great Britain and Northern Ireland.

In [15]:
installed_cap_entsoe = pd.read_csv('../data/raw/ENTSO-E_TP/ENTSO-E_TP_installed_cap.csv', 
                                   index_col=[0,1]).T
installed_cap_entsoe.to_csv('../data/intermediate/Installed_cap_ENTSO-E_TP.csv')

In [13]:
patched_areas = ['GB', 'GB-NIR']
wind_techs = ['Wind Onshore', 'Wind Offshore']
wind_cap_entsoe = installed_cap_entsoe.loc[:, (wind_techs, patched_areas)]
wind_cap_entsoe.index = pd.DatetimeIndex(pd.to_datetime(wind_cap_entsoe.index), 
                                         name='Date')
wind_cap_patched = pd.concat([wind_cap, wind_cap_entsoe], axis=1).sort_index(1)
wind_cap_patched.loc[:, (wind_techs, patched_areas)]  # Checking

Wind Offshore        Wind Onshore       
Country code            GB GB-NIR           GB GB-NIR
Date                                                 
2018-01-01          6071.0    NaN      12144.0  995.0
2019-01-01          9379.0    NaN      12638.0  995.0

Combine on- and offshore capacities for some countries because no offshore generation data is available.

In [7]:
no_offshore_gen_data = ['FI']
wind_cap_fixed = wind_cap_patched.copy()
wind_cap_fixed.loc[:, ('Wind Onshore', no_offshore_gen_data)] = (
    wind_cap.loc[:, (['Wind Onshore', 'Wind Offshore'], no_offshore_gen_data)].sum(axis=1, level=1).values
)
wind_cap_fixed.loc[:, ('Wind Offshore', no_offshore_gen_data)] = 0
wind_cap_fixed.loc[:, (slice(None), no_offshore_gen_data)]

,Wind Offshore,Wind Onshore
Country code,FI,FI
Date,,
2018-01-01,0,2044.0
2019-01-01,0,2041.0


ERCOT data for Texas

In [8]:
for gentype, sheet_name in zip(['Wind Onshore', 'Solar'], ['Wind Chart', 'Solar Chart']):
    df = pd.read_excel('../data/external/IEA data/Capacity_Changes_by_Fuel_Type_Charts_March_2019.xlsx',
                       sheet_name=sheet_name, skiprows=44, 
                       usecols=['Year', 'Cumulative MW Installed'],
                       ).dropna()
    df.Year = df.Year.astype(int)
    df['Date'] = pd.to_datetime({'year': df.Year + 1, 'month': 1, 'day': 1})
    ts = df.set_index('Date').query("Year in (2017, 2018)")['Cumulative MW Installed']
    wind_cap_fixed[(gentype, 'US-TX')] = ts
wind_cap_fixed

Wind Offshore                                                  \
Country code            AT      BE       DE       DK    ES FI   FR      GB   
Date                                                                         
2018-01-01             0.0   876.5  5411.36  1267.34   5.0  0  0.0  6071.0   
2019-01-01             0.0  1185.5  6380.16  1328.54  10.0  0  2.2  9379.0   

                           ... Wind Onshore                                   \
Country code GB-NIR    IE  ...           GB GB-NIR       IE       IT      NL   
Date                       ...                                                 
2018-01-01      NaN  25.2  ...      12144.0  995.0  3345.61  9505.97  3259.0   
2019-01-01      NaN  25.2  ...      12638.0  995.0  3538.96  9957.97  3353.0   

                                                 Solar  
Country code       NO      PT      SE    US-TX   US-TX  
Date                                                    
2018-01-01    1192.55  5326.6  6498.5  20682.0  1068.7  
2019-01-01    1672.95  5380.0  7215.0  21751.0  1857.7  

[2 rows x 32 columns]

Preview the capacity data for selected countries _at the end of each year_.

In [9]:
wind_cap_selected = wind_cap_fixed.loc[:, (slice(None), selected_country_codes)].T.dropna()
wind_cap_selected

Date                        2018-01-01  2019-01-01
              Country code                        
Wind Offshore AT                 0.000       0.000
              BE               876.500    1185.500
              DE              5411.360    6380.160
              DK              1267.340    1328.540
              ES                 5.000      10.000
              FI                 0.000       0.000
              FR                 0.000       2.200
              GB              6071.000    9379.000
              IE                25.200      25.200
              IT                 0.000       0.000
              NL              1117.800    1117.800
              NO                 2.300       2.300
              PT                 0.000       0.000
              SE               201.650     191.700
Wind Onshore  AT              2843.960    3044.960
              BE              1970.950    2174.750
              DE             50777.930   52930.930
              DK              4222.309    4429.809
              ES             23092.000   23484.000
              FI              2044.000    2041.000
              FR             13757.100   15307.250
              GB             12144.000   12638.000
              GB-NIR           995.000     995.000
              IE              3345.610    3538.960
              IT              9505.970    9957.970
              NL              3259.000    3353.000
              NO              1192.550    1672.950
              PT              5326.600    5380.000
              SE              6498.500    7215.000
              US-TX          20682.000   21751.000
Solar         US-TX           1068.700    1857.700

## Read installed capacity data for solar power

In [10]:
_df = (pd.read_csv('../data/raw/ENTSO-E_TP/ENTSO-E_TP_installed_cap.csv', index_col=[0, 1])
         .loc[['Solar']]
         .dropna())
_df.columns = pd.to_datetime(_df.columns)
_df.columns.name = 'Date'
solar_cap = _df
solar_cap

Date          2018-01-01  2019-01-01
Solar AT          1193.0      1193.0
      BE          2953.0      3369.0
      DE         42804.0     45299.0
      DK          1002.0      1014.0
      ES          6722.0      6751.0
      FR          7170.0      8188.0
      GB         12471.0     13346.0
      GB-NIR        92.0        92.0
      IT          4719.0      4717.0
      NL          2584.0      3937.0
      PT           272.0       324.0

Write to disk for later use.

In [11]:
installed_cap = pd.concat([wind_cap_selected, solar_cap], axis=0).T.sort_index(1)
installed_cap.to_csv('../data/intermediate/Installed_cap.csv')